### !pip install torch==1.0.0 torchvision==0.2.2 -f https://download.pytorch.org/whl/cu90/torch_stable.html
%load_ext autoreload
%autoreload 2

import argparse
import os
import torch
import capsulenet

# setting the hyper parameters
parser = argparse.ArgumentParser(description="Capsule Network.")

parser.add_argument('--epochs', default=50, type=int)
parser.add_argument('--batch_size', default=200, type=int)
parser.add_argument('--lr', default=0.001, type=float,help="Initial learning rate")
parser.add_argument('--lr_decay', default=0.9, type=float,help="The value multiplied by lr at each epoch. Set a larger value for larger epochs")
parser.add_argument('--lam_recon', default=0.0005 * 784, type=float,help="The coefficient for the loss of decoder")
parser.add_argument('-r', '--routings', default=3, type=int,help="Number of iterations used in routing algorithm. should > 0")  # num_routing should > 0
parser.add_argument('--shift_pixels', default=2, type=int,help="Number of pixels to shift at most in each direction.")
parser.add_argument('--data_dir', default='./data/',help="Directory of data. If no data, use \'--download\' flag to download it")
parser.add_argument('--download', action='store_true',help="Download the required data.")
parser.add_argument('--save_dir', default='./result')
parser.add_argument('-t', '--testing', action='store_true',help="Test the trained model on testing dataset")
parser.add_argument('-w', '--weights', default=None,help="The path of the saved weights. Should be specified when testing")
args = parser.parse_args(args=[])
print(args)

if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

# load data
loc=0
#train_loader, test_loader = capsulenet.load_mnist(True,args.data_dir, download=False, batch_size=args.batch_size,location=0 )
train_loader, test_loader = capsulenet.load_mnist(False,'./data/', download=False, batch_size=args.batch_size, location=loc)

# define model
#model = capsulenet.CapsuleNet(input_size=[1, 28, 28], classes=10, routings=5)
model = capsulenet.CapsuleNet(input_size=[1, 27, 9], classes=5, routings=5)
if torch.cuda.is_available():
  model.cuda()
print(model)


# train or test
if args.weights is not None:  # init the model weights with provided one
    model.load_state_dict(torch.load(args.weights))
if not args.testing:
    capsulenet.train(model, train_loader, test_loader, args)
else:  # testing
    if args.weights is None:
        print('No weights are provided. Will test using random initialized weights.')
    test_loss, test_acc = test(model=model, test_loader=test_loader, args=args)
    print('test acc = %.4f, test loss = %.5f' % (test_acc, test_loss))
    #show_reconstruction(model, test_loader, 50, args)
    
torch.save(model.state_dict(), 'trained_model'+str(loc)+'.pt')

In [11]:
### !pip install torch==1.0.0 torchvision==0.2.2 -f https://download.pytorch.org/whl/cu90/torch_stable.html
%load_ext autoreload
%autoreload 2

import argparse
import os
import torch
import capsulenet

# setting the hyper parameters
parser = argparse.ArgumentParser(description="Capsule Network.")

parser.add_argument('--epochs', default=50, type=int)
parser.add_argument('--batch_size', default=200, type=int)
parser.add_argument('--lr', default=0.001, type=float,help="Initial learning rate")
parser.add_argument('--lr_decay', default=0.9, type=float,help="The value multiplied by lr at each epoch. Set a larger value for larger epochs")
parser.add_argument('--lam_recon', default=0.0005 * 784, type=float,help="The coefficient for the loss of decoder")
parser.add_argument('-r', '--routings', default=3, type=int,help="Number of iterations used in routing algorithm. should > 0")  # num_routing should > 0
parser.add_argument('--shift_pixels', default=2, type=int,help="Number of pixels to shift at most in each direction.")
parser.add_argument('--data_dir', default='./data/',help="Directory of data. If no data, use \'--download\' flag to download it")
parser.add_argument('--download', action='store_true',help="Download the required data.")
parser.add_argument('--save_dir', default='./result')
parser.add_argument('-t', '--testing', action='store_true',help="Test the trained model on testing dataset")
parser.add_argument('-w', '--weights', default=None,help="The path of the saved weights. Should be specified when testing")
args = parser.parse_args(args=[])
print(args)

if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

# load data
loc=2
#train_loader, test_loader = capsulenet.load_mnist(True,args.data_dir, download=False, batch_size=args.batch_size,location=0 )
train_loader, test_loader = capsulenet.load_mnist(False,'./data/', download=False, batch_size=args.batch_size, location=loc)

# define model
#model = capsulenet.CapsuleNet(input_size=[1, 28, 28], classes=10, routings=5)
model = capsulenet.CapsuleNet(input_size=[2, 9, 9], classes=5, routings=5)
# if torch.cuda.is_available():
#   model.cuda()
print(model)


# train or test
if args.weights is not None:  # init the model weights with provided one
    model.load_state_dict(torch.load(args.weights))
if not args.testing:
    capsulenet.train(model, train_loader, test_loader, args)
else:  # testing
    if args.weights is None:
        print('No weights are provided. Will test using random initialized weights.')
    test_loss, test_acc = test(model=model, test_loader=test_loader, args=args)
    print('test acc = %.4f, test loss = %.5f' % (test_acc, test_loss))
    #show_reconstruction(model, test_loader, 50, args)
    
torch.save(model.state_dict(), 'trained_model'+str(loc)+'.pt')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Namespace(batch_size=200, data_dir='./data/', download=False, epochs=50, lam_recon=0.392, lr=0.001, lr_decay=0.9, routings=3, save_dir='./result', shift_pixels=2, testing=False, weights=None)
torch.Size([3659, 18, 9])
torch.Size([3659, 22])
tensor([0., 0., 0., 0., 0., nan, nan, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


IndexError: tensors used as indices must be long, byte or bool tensors

In [20]:
z = y[:, None]
z = torch.zeros(z.size(0), 3).scatter_(1, z, 1)
print(z.shape)

z.shape torch.Size([132, 1])
torch.Size([132, 3])


In [51]:

# train or test
if args.weights is not None:  # init the model weights with provided one
    model.load_state_dict(torch.load(args.weights))
if not args.testing:
    capsulenet.train(model, train_loader, test_loader, args)
else:  # testing
    if args.weights is None:
        print('No weights are provided. Will test using random initialized weights.')
    test_loss, test_acc = test(model=model, test_loader=test_loader, args=args)
    print('test acc = %.4f, test loss = %.5f' % (test_acc, test_loss))
    #show_reconstruction(model, test_loader, 50, args)

Begin Training----------------------------------------------------------------------
0
==> Epoch 00: loss=0.47787, val_loss=0.38342, val_acc=0.9107, time=298s
best val_acc increased to 0.9107
1
==> Epoch 01: loss=0.34146, val_loss=0.34274, val_acc=0.9112, time=296s
best val_acc increased to 0.9112
2
==> Epoch 02: loss=0.32838, val_loss=0.33054, val_acc=0.9107, time=297s
3
==> Epoch 03: loss=0.32035, val_loss=0.32142, val_acc=0.9126, time=295s
best val_acc increased to 0.9126
4
==> Epoch 04: loss=0.31328, val_loss=0.31984, val_acc=0.9129, time=295s
best val_acc increased to 0.9129
5
==> Epoch 05: loss=0.30858, val_loss=0.31849, val_acc=0.9117, time=300s
6
==> Epoch 06: loss=0.30585, val_loss=0.31476, val_acc=0.9120, time=297s
7
==> Epoch 07: loss=0.30104, val_loss=0.31097, val_acc=0.9141, time=301s
best val_acc increased to 0.9141
8
==> Epoch 08: loss=0.29779, val_loss=0.30894, val_acc=0.9142, time=321s
best val_acc increased to 0.9142
9
==> Epoch 09: loss=0.29544, val_loss=0.30800, val

In [53]:
torch.save(model.state_dict(), 'trained_model1.pt')

NameError: ignored

In [18]:
import torch
from torch.utils.data import TensorDataset,DataLoader
import numpy as np
import os
from scipy import stats

path="./data/"
fieldname = '_19790101-20190228.npy'
x1_arr = np.load(path+'z1000' + fieldname)  # geopotential height data (9*9 resolution)
x2_arr = np.load(path+'pv300' + fieldname)  # potential vorticity data (9*9 resolution)

#x3_arr = np.load(path+'z500' + fieldname)  # geopotential height data (9*9 resolution)

#x1_arr_flat = stats.zscore(x1_arr.reshape([x1_arr.shape[0], x1_arr.shape[1] * x1_arr.shape[2]]))
#x2_arr_flat = stats.zscore(x2_arr.reshape([x2_arr.shape[0], x2_arr.shape[1] * x2_arr.shape[2]]))
#x3_arr_flat = stats.zscore(x2_arr.reshape([x3_arr.shape[0], x3_arr.shape[1] * x3_arr.shape[2]]))
y_arr = np.load(path+'rain'+fieldname)  # rain data
print(y_arr.shape)
tensor_x = torch.Tensor(np.concatenate([x1_arr, x2_arr], axis=1))
print(tensor_x.shape)
tensor_y = torch.Tensor(y_arr)
print(tensor_y.shape)

forecast_dataset = TensorDataset(tensor_x,tensor_y) # creates a dataset based on tensors
forecast_dataset2 = []


for j in forecast_dataset:
  forecast_dataset2.append( ( j[0].reshape((3, 9, 9)) , j[1] ) ) # Rotem: ask about order of 162
training_ds, validation_ds = torch.utils.data.random_split(forecast_dataset2, [2195,1464])
training_dataloader = DataLoader(training_ds,batch_size=200,shuffle=True)
valid_dataloader = DataLoader(validation_ds,batch_size=200)

for batch_x,batch_y in training_dataloader:
  print(batch_x.shape, batch_y.shape)

(3659, 9, 9)
(3659, 22)
torch.Size([3659, 18, 9])
torch.Size([3659, 22])


RuntimeError: shape '[3, 9, 9]' is invalid for input of size 162

In [21]:
  loss = caps_loss(batch_y, batch_y, batch_x,batch_x, args.lam_recon)

  loss

tensor(-7271.9624)